In [ ]:
from keras import layers, models
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

# 분산 방식 Modeling - OOP

In [ ]:

class AE_Model_OOP(models.Model):
    def __init__(self, x_nodes=784, z_dim=36):
        x = layers.Input(shape=(x_nodes,))
        z = layers.Dense(z_dim, activation='relu')(x)
        y = layers.Dense(x_nodes, activation='sigmoid')(z)

        super().__init__(x, y)

        self.x = x
        self.z = z
        self.z_dim = z_dim

        self.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

    def encoder(self):
        return models.Model(self.x, self.z)

    def decoder(self):
        z = layers.Input(shape=(self.z_dim,))
        y_layer = self.layers[-1]
        y = y_layer(z)
        return models.Model(z, y)

# Data - MNIST

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Training and Evaluation

In [ ]:
def show_ae(autoencoder):
    encoder = autoencoder.encoder()
    decoder = autoencoder.decoder()

    encoded_imgs = encoder.predict(x_test)
    decoded_imgs = decoder.predict(encoded_imgs)

    n = 10
    plt.figure(figsize=(20, 6))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1)
        plt.imshow(x_test[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(3, n, i + 1 + n)
        plt.stem(encoded_imgs[i].reshape(-1))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(3, n, i + 1 + n + n)
        plt.imshow(decoded_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

# Draw Result

In [ ]:
def plot_loss(history, title=None):
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    if title is not None:
        plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Training', 'Validation'], loc=0)
    plt.show()

def plot_acc(history, title=None):
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    if title is not None:
        plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Training', 'Validation'], loc=0)
    plt.show()

# Usage

In [ ]:
INPUT_NODES = 784
Z_DIMENSION = 36

autoencoder = AE_Model_OOP(INPUT_NODES, Z_DIMENSION)

history = autoencoder.fit(x_train, x_train, epochs=10, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

plot_acc(history)
plot_loss(history)
show_ae(autoencoder)